# MongoDB Joins

In MongoDB joins happen within the context of the aggregate pipeline.

Unlike relational databases where there is a variety of joins to utilize, MongoDB has only left outer equi-joins at it's disposal.

The most basic form of this syntax looks like so:

```json
db.COLLECTION_NAME.aggregate([{
    $lookup: {
        from: "COLLECTION_TO_JOIN",
        localField: "localField",
        foreignField: "foreignField",
        as: "jsonKey"
    }
}]);
```

## Example 1 -  A Simple Join Using Foreign Keys

1. Let's get our collections

In [ ]:
from pymongo import MongoClient
from pprint import pprint

import pymongo

# Initialize a Mongo Client
#################################################
# Update UPDATE-ME in the connection code
#################################################
# Client 1 - mongodb-1.dsa.missouri.edu
# Client 2 - mongodb-2.dsa.missouri.edu
# Client 3 - mongodb-3.dsa.missouri.edu
# Client 4 - mongodb-4.dsa.missouri.edu
#################################################
#
client = MongoClient('UPDATE-ME',
                     username='mlLargeReader',
                     password='mlLargeReader.create.role',
                     authSource='mlLarge')

db = client.mlLarge

# Get list of collections
cursor = db.collection_names()
for document in cursor:
    pprint(document)

2. Let's join two collections:

In [ ]:
documents = db.movies.aggregate([{
    # Let's bring in the links collection using movieId as our key
    "$lookup": {
        "from": "links",
        "localField": "movieId",
        "foreignField": "movieId",
        "as": "links"
    }
},{
    # Let's limit ourselves to something reasonable, just one document for now
    "$limit": 1
}]);

for document in documents:
    pprint(document)

**Note:** The joined tables are created as children of the root object, in Mongo v3.6+ you can use the $mergeObjects directive to merge them with the root object.

## Example 2 - Multiple Joins

In [ ]:
documents = db.movies.aggregate([{
    # Let's join the ratings collection using movieId
    "$lookup": {
        "from": "ratings",
        "localField": "movieId",
        "foreignField": "movieId",
        "as": "ratings"
    }
}, {
    # Specify that we only want movies that have ratings
    "$match": {
        "ratings": {
            "$ne": []
        }
    }
}, {
    # Limit ourselves to one movie
    "$limit": 1
}, {
    # Bring in the links for our movie
    "$lookup": {
        "from": "links",
        "localField": "movieId",
        "foreignField": "movieId",
        "as": "links"
    }
}]);
#{
#     "$unwind": "$ratings"
# },{
#     "$limit": 3
# }]);

for document in documents:
    pprint(document)

## Example 3 - Joins with filtering

**Note:** In version 3.6+ of MongoDB you can pass an aggregate pipeline to the `$lookup` directive allowing you to filter those results directly, this is not possible in our version (3.4.15) of mongo.

In [ ]:
documents = db.movies.aggregate([{
    # Let's join the ratings collection using movieId
    "$lookup": {
        "from": "ratings",
        "localField": "movieId",
        "foreignField": "movieId",
        "as": "ratings"
    }
}, {
    # Specify that we only want movies that have ratings
    "$match": {
        "ratings": {
            "$ne": []
        }
    }
}, {
    # Limit ourselves to one movie
    "$limit": 1
}, {
    # Bring in the links for our movie
    "$lookup": {
        "from": "links",
        "localField": "movieId",
        "foreignField": "movieId",
        "as": "links"
    }
}, {
    # Filter the ratings and specify our columns.
    # Note: There is no '*' specifier as in PostgresQL and most relational databases
    # you must explicitly list all desired columns if you project any of them.
    "$project": {
        "movieId": 1,
        "title": 1,
        "genres": 1,
        "links": 1,
        "ratings": {
            "$filter": {
                "input": "$ratings",
                "as": "rating",
                "cond": {
                    "$gte": ["$$rating.rating", 4.0]
                }
            },
        }
    }
}]);

for document in documents:
    pprint(document)

## <span style='background:yellow'>Your Turn</span>

In [ ]:
## Create a query that returns a single movie with it's tags 
## ---------------------------------------------------------

documents = db.movies.aggregate([{
    # Let's bring in the links collection using movieId as our key
    "$lookup": {
        "from": "tags",
        "localField": "movieId",
        "foreignField": "movieId",
        "as": "tags"
    }
},{
    # Let's limit ourselves to something reasonable, just one document for now
    "$limit": 1
}]);


for document in documents:
    pprint(document)

In [ ]:
## Create a query that returns the movie and it's tags, but only if it has a rating, limit your query to 5 results.
## ---------------------------------------------------------


documents = db.movies.aggregate([{
    "$lookup": {
        "from": "tags",
        "localField": "movieId",
        "foreignField": "movieId",
        "as": "tags"
    }
},{
    "$limit": 5
}, { "$lookup": {
        "from": "ratings",
        "localField": "movieId",
        "foreignField": "movieId",
        "as": "ratings"
    }
}, {
    "$match": {
        "ratings": {
            "$ne": []
        }
    }
}]);

for document in documents:
    pprint(document)

In [ ]:
# Be sure to run this cell when you are finished. Thank you.
client.close()

# Save your notebook, then `File > Close and Halt`

---